In [7]:
from cellshape_cloud.lightning_autoencoder import CloudAutoEncoderPL
from cellshape_cloud.cloud_autoencoder import CloudAutoEncoder
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
# define the dataset location
from scipy.stats import zscore
# First with nine classes
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score, average_precision_score, f1_score,roc_auc_score, precision_score,recall_score


from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score

from sklearn.model_selection import cross_val_score
from sklearn import svm

from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
import pandas as pd
import torch
import sys 
import easydict

sys.path.append('/home/mvries/Documents/GitHub/UnsupervisedPointCloudReconstruction/')

from dataset import Dataset


In [8]:
path = '/run/user/1128299809/gvfs/smb-share:server=rds.icr.ac.uk,' \
    'share=data/DBI/DUDBI/DYNCESYS/mvries/ResultsAlma/ShapeNet/' 

In [16]:
from pathlib import Path
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import os
import h5py
import numpy as np
from glob import glob
from sklearn.svm import LinearSVC
import pandas as pd

all_results = {}
for encoder in ['foldingnet', 'dgcnn']:
    for shape in ['plane', 'gaussian', 'sphere']:
    
        model_path = Path(path + f"{encoder}_foldingnet_512_{shape}_001/ShapeNetExperiment/")
        for k_mod in model_path.iterdir():
            
            new_mod_path = k_mod / "checkpoints/last.ckpt"
            checkpoint = torch.load(new_mod_path, map_location=lambda storage, loc: storage)
            
            k = int(str(k_mod.name)[-2:])
            model = CloudAutoEncoder(
                num_features=512,
                k=k,
                encoder_type=encoder,
                decoder_type='foldingnet',
                shape=shape,
                gaussian_path = '/home/mvries/Documents/GitHub/cellshape-cloud/cellshape_cloud/vendor/gaussian.npy',
                sphere_path = '/home/mvries/Documents/GitHub/cellshape-cloud/cellshape_cloud/vendor/sphere.npy'
                )
            args = easydict.EasyDict(
                {"learning_rate_autoencoder": 0.0001,
                   "encoder_type": encoder,
                   "decoder_type": "foldingnet",
                   "k": k,
                "dataframe_path": "",
                "cloud_dataset_path": "",
                "batch_size": 1,
                "pretrained_path": "",
                "dataset_type": "Other"})
            model = CloudAutoEncoderPL(args=args, model=model)

            model.load_state_dict(checkpoint['state_dict'])

            model.cuda()
            
            train_dset = Dataset(root='/mnt/nvme0n1/Datasets', split='train')
            test_dset = Dataset(root='/mnt/nvme0n1/Datasets', split='test')

            train_dloader = DataLoader(
                    train_dset, batch_size=1, shuffle=False, num_workers=0
                )
            test_dloader = DataLoader(
                    test_dset, batch_size=1, shuffle=False, num_workers=0
                )
            
            all_feats = []
            all_outs = []
            all_treats = []
            all_serials = []
            all_inputs = []



            for data in tqdm(train_dloader):
                inp = data[0]
                all_inputs.append(torch.squeeze(inp).numpy())
                inputs = inp.cuda()
                treatment = data[1]

    
                outputs, features = model.model(inputs)
                all_feats.append(torch.squeeze(features).detach().cpu().numpy())
                all_outs.append(torch.squeeze(outputs).detach().cpu().numpy())
                all_treats.append(int(treatment[0].numpy()))
                
            all_feats_test = []
            all_outs_test = []
            all_treats_test = []
            all_serials_test = []
            all_inputs_test = []



            for data in tqdm(test_dloader):
                inp = data[0]
                all_inputs_test.append(torch.squeeze(inp).numpy())
                inputs = inp.cuda()
                treatment = data[1]

    
                outputs, features = model.model(inputs)
                all_feats_test.append(torch.squeeze(features).detach().cpu().numpy())
                all_outs_test.append(torch.squeeze(outputs).detach().cpu().numpy())
                all_treats_test.append(int(treatment[0].numpy()))
                
            train_feats = np.asarray(all_feats)
            train_labs = all_treats

            test_feats = np.asarray(all_feats_test)
            test_labs = all_treats_test
            
            # label encode the target variable


            X_train, X_test, y_train, y_test = train_feats, test_feats, train_labs, test_labs

            clf = LinearSVC(random_state=0) 

            clf.fit(X_train, y_train)

            y_pred = clf.predict(X_test)
#             y_proba = clf.predict_proba(X_test)

            acc_cls = accuracy_score(y_test, y_pred)
#             bal_acc_cls = balanced_accuracy_score(y_test, y_pred)
#             f1_cls = f1_score(y_test, y_pred, average='macro')
#             # auc_cls_ovo = roc_auc_score(y_test, y_proba, average = 'macro')
#             auc_cls_ovr = roc_auc_score(y_test, y_proba, average = 'macro', multi_class='ovr')
#             precision_cls = precision_score(y_test, y_pred, average='macro')
#             recall_cls = recall_score(y_test, y_pred, average='macro')

            results_dl = {'acc': acc_cls,
#                  'bal_acc': bal_acc_cls,
#                  'f1': f1_cls,
# #                  'auc_ovo': auc_cls_ovo,
#                   'auc_ovr': auc_cls_ovr,
#                  'precision': precision_cls,
#                  'recall': recall_cls
                         }
            
            print(f"Results for encoder {encoder} shape {shape} k {k}.")
            print(results_dl)
            all_results[f"{encoder}_{shape}_k{k}"] = results_dl

#             fig, ax = plt.subplots(figsize=(10, 10))
#             cm = plot_confusion_matrix(clf, X_test, y_test, normalize='true', ax=ax, cmap=plt.cm.plasma)  
#             cm.ax_.get_images()[0].set_clim(0, 1)
#             plt.show()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2468/2468 [00:10<00:00, 235.78it/s]


Results for encoder foldingnet shape plane k 20.
{'acc': 0.8703403565640194}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2468/2468 [00:08<00:00, 288.83it/s]


Results for encoder foldingnet shape plane k 40.
{'acc': 0.8687196110210696}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2468/2468 [00:06<00:00, 386.55it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2468/2468 [00:37<00:00, 65.52it/s]
/home/mvries/anaconda3/envs/dcc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Results for encoder dgcnn shape sphere k 20.
{'acc': 0.8780388978930308}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2468/2468 [01:23<00:00, 29.73it/s]


Results for encoder dgcnn shape sphere k 40.
{'acc': 0.8788492706645057}


/home/mvries/anaconda3/envs/dcc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [15]:
clf = LinearSVC(random_state=0) 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
#             y_proba = clf.predict_proba(X_test)

acc_cls = accuracy_score(y_test, y_pred)
#             bal_acc_cls = balanced_accuracy_score(y_test, y_pred)
#             f1_cls = f1_score(y_test, y_pred, average='macro')
#             # auc_cls_ovo = roc_auc_score(y_test, y_proba, average = 'macro')
#             auc_cls_ovr = roc_auc_score(y_test, y_proba, average = 'macro', multi_class='ovr')
#             precision_cls = precision_score(y_test, y_pred, average='macro')
#             recall_cls = recall_score(y_test, y_pred, average='macro')

results_dl = {'acc': acc_cls,
#                  'bal_acc': bal_acc_cls,
#                  'f1': f1_cls,
# #                  'auc_ovo': auc_cls_ovo,
#                   'auc_ovr': auc_cls_ovr,
#                  'precision': precision_cls,
#                  'recall': recall_cls
                         }
           
print(f"Results for encoder {encoder} shape {shape} k {k}.")
print(results_dl)
all_results[f"{encoder}_{shape}_k{k}"] = results_dl

Results for encoder foldingnet shape plane k 20.
{'acc': 0.8545380875202593}


/home/mvries/anaconda3/envs/dcc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,acc,bal_acc,f1,auc_ovr,precision,recall
foldingnet_plane_k20,0.860211,0.829477,0.814972,0.994621,0.810845,0.829477
foldingnet_plane_k40,0.860616,0.819808,0.808378,0.994197,0.808139,0.819808
foldingnet_gaussian_k40,0.860616,0.818395,0.810548,0.994786,0.812245,0.818395
foldingnet_gaussian_k20,0.868720,0.833802,0.824146,0.995365,0.826508,0.833802
foldingnet_sphere_k40,0.862642,0.824052,0.813014,0.994438,0.810090,0.824052
foldingnet_sphere_k20,0.861021,0.835093,0.821021,0.994983,0.820443,0.835093
dgcnn_plane_k40,0.882496,0.850680,0.844484,0.995304,0.845003,0.850680
dgcnn_plane_k20,0.876823,0.834506,0.828420,0.995787,0.828524,0.834506
dgcnn_gaussian_k40,0.874392,0.833302,0.827238,0.995940,0.831589,0.833302
dgcnn_gaussian_k20,0.871961,0.831855,0.826124,0.995962,0.829962,0.831855


In [59]:
all_results

{'foldingnet_plane_k20': {'acc': 0.8602106969205835,
  'bal_acc': 0.8294767441860464,
  'f1': 0.81497229838644,
  'auc_ovr': 0.9946212557499244,
  'precision': 0.8108446413111394,
  'recall': 0.8294767441860464},
 'foldingnet_plane_k40': {'acc': 0.8606158833063209,
  'bal_acc': 0.8198081395348839,
  'f1': 0.8083783383012861,
  'auc_ovr': 0.9941972343765089,
  'precision': 0.8081390021500982,
  'recall': 0.8198081395348839},
 'foldingnet_gaussian_k40': {'acc': 0.8606158833063209,
  'bal_acc': 0.8183953488372093,
  'f1': 0.8105477579992784,
  'auc_ovr': 0.9947861777123432,
  'precision': 0.812245190710812,
  'recall': 0.8183953488372093},
 'foldingnet_gaussian_k20': {'acc': 0.8687196110210696,
  'bal_acc': 0.8338023255813954,
  'f1': 0.8241456078006477,
  'auc_ovr': 0.9953650116468189,
  'precision': 0.8265075190324239,
  'recall': 0.8338023255813954},
 'foldingnet_sphere_k40': {'acc': 0.8626418152350082,
  'bal_acc': 0.8240523255813954,
  'f1': 0.8130136219395248,
  'auc_ovr': 0.9944384

In [13]:
import pandas as pd

# Create a dictionary with the data
data = {
    'foldingnet_plane_k20': {'acc': 0.8602106969205835, 'bal_acc': 0.8294767441860464, 'f1': 0.81497229838644, 'auc_ovr': 0.9946212557499244, 'precision': 0.8108446413111394, 'recall': 0.8294767441860464},
    'foldingnet_plane_k40': {'acc': 0.8606158833063209, 'bal_acc': 0.8198081395348839, 'f1': 0.8083783383012861, 'auc_ovr': 0.9941972343765089, 'precision': 0.8081390021500982, 'recall': 0.8198081395348839},
    'foldingnet_gaussian_k40': {'acc': 0.8606158833063209, 'bal_acc': 0.8183953488372093, 'f1': 0.8105477579992784, 'auc_ovr': 0.9947861777123432, 'precision': 0.812245190710812, 'recall': 0.8183953488372093},
    'foldingnet_gaussian_k20': {'acc': 0.8687196110210696, 'bal_acc': 0.8338023255813954, 'f1': 0.8241456078006477, 'auc_ovr': 0.9953650116468189, 'precision': 0.8265075190324239, 'recall': 0.8338023255813954},
    'foldingnet_sphere_k40': {'acc': 0.8626418152350082, 'bal_acc': 0.8240523255813954, 'f1': 0.8130136219395248, 'auc_ovr': 0.9944384189802697, 'precision': 0.8100896397166437, 'recall': 0.8240523255813954},
    'foldingnet_sphere_k20': {'acc': 0.8610210696920584, 'bal_acc': 0.835093023255814, 'f1': 0.8210209658862204, 'auc_ovr': 0.9949832601330015, 'precision': 0.8204432318070088, 'recall': 0.835093023255814},
    'dgcnn_plane_k40': {'acc': 0.8824959481361426, 'bal_acc': 0.8506802325581395, 'f1': 0.8444841515255911, 'auc_ovr': 0.9953039809073992, 'precision': 0.8450030332442896, 'recall': 0.8506802325581395},
    'dgcnn_plane_k20': {'acc': 0.8768233387358185, 'bal_acc': 0.8345058139534884, 'f1': 0.8284199320282486, 'auc_ovr': 0.9957873224150295, 'precision': 0.828524235796446, 'recall': 0.8345058139534884},
    'dgcnn_gaussian_k40': {'acc': 0.8743922204213939, 'bal_acc': 0.8333023255813954, 'f1': 0.8272380830521218, 'auc_ovr': 0.9959398475294865, 'precision': 0.8315889914115943, 'recall': 0.8333023255813954},
    'dgcnn_gaussian_k20': {'acc': 0.8719611021069692, 'bal_acc': 0.8318546511627906, 'f1': 0.8261236535111103, 'auc_ovr': 0.9959615077722421, 'precision': 0.8299624645945757, 'recall': 0.8318546511627906},
    'dgcnn_sphere_k20': {'acc': 0.8772285251215559, 'bal_acc': 0.8417732558139533, 'f1': 0.8326188759007765, 'auc_ovr': 0.9961078716586498, 'precision': 0.8315546685638058, 'recall': 0.8417732558139533},
    'dgcnn_sphere_k40': {'acc': 0.873581847649919, 'bal_acc': 0.8302965116279071, 'f1': 0.8248064106717233, 'auc_ovr': 0.9950061620695319, 'precision': 0.8322898363970854, 'recall': 0.8302965116279071}
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Round the numbers to two decimal places
df = df.round(3)

# Transpose the DataFrame
df = df.transpose()

# Get the column names and maximum values
column_names = df.columns
max_values = df.max()

# Create a LaTeX table
latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{|l|" + "c|" * len(column_names) + "}\n\\hline\n"

# Add column headers
latex_table += " & ".join([''] + column_names) + " \\\\\n\\hline\n"

# Add rows with highlighting for maximum values
for index, row in df.iterrows():
    row_values = [f"\\textbf{{{value}}}" if value == max_values[column] else str(value) for column, value in row.items()]
    latex_table += " & ".join([index] + row_values) + " \\\\\n"

# Add table footer
latex_table += "\\hline\n\\end{tabular}\n\\end{table}"

# Print the LaTeX table
print(latex_table)


\begin{table}[h]
\centering
\begin{tabular}{|l|c|c|c|c|c|c|}
\hline
acc & bal_acc & f1 & auc_ovr & precision & recall \\
\hline
foldingnet_plane_k20 & 0.86 & 0.829 & 0.815 & 0.995 & 0.811 & 0.829 \\
foldingnet_plane_k40 & 0.861 & 0.82 & 0.808 & 0.994 & 0.808 & 0.82 \\
foldingnet_gaussian_k40 & 0.861 & 0.818 & 0.811 & 0.995 & 0.812 & 0.818 \\
foldingnet_gaussian_k20 & 0.869 & 0.834 & 0.824 & 0.995 & 0.827 & 0.834 \\
foldingnet_sphere_k40 & 0.863 & 0.824 & 0.813 & 0.994 & 0.81 & 0.824 \\
foldingnet_sphere_k20 & 0.861 & 0.835 & 0.821 & 0.995 & 0.82 & 0.835 \\
dgcnn_plane_k40 & \textbf{0.882} & \textbf{0.851} & \textbf{0.844} & 0.995 & \textbf{0.845} & \textbf{0.851} \\
dgcnn_plane_k20 & 0.877 & 0.835 & 0.828 & \textbf{0.996} & 0.829 & 0.835 \\
dgcnn_gaussian_k40 & 0.874 & 0.833 & 0.827 & \textbf{0.996} & 0.832 & 0.833 \\
dgcnn_gaussian_k20 & 0.872 & 0.832 & 0.826 & \textbf{0.996} & 0.83 & 0.832 \\
dgcnn_sphere_k20 & 0.877 & 0.842 & 0.833 & \textbf{0.996} & 0.832 & 0.842 \\
dgcnn_sphere_k4